## IRRetriever

In [ ]:
import warnings
import logging

from retrievers import IRRetriever

logging = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

retriever = IRRetriever(ir_model="<YOUR_MODEL>", top_k=5)
docs = retriever._get_relevant_documents("<YOUR_QUESTION>")

## IRChain: without memory

In [2]:
from chains import IRChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate.from_template(
    "Title {title}\n{page_content}"
)

system_template = """You are an assistant chatbot. Answer the best as you can."""

human_template = """Below are documents provided.
{context}

-------------
Question: {question}
Answer:"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])  

# Initialize llm and KeylookChain

# You can add streaming callbacks to llm model
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat_model = ChatOpenAI(
    openai_api_key = "<YOUR_KEY>",
    # streaming=True,
    # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# As IRChain inherits from LLMChain, it has the same interface as it.
# That means, you can add callbacks, output_parser, or any other modules
# provided through langchain.
from langchain.callbacks import StdOutCallbackHandler

chain = IRChain(
    prompt = prompt,
    document_prompt = document_prompt,
    llm = chat_model,
    retriever = retriever,
    document_variable_name = "context",
    document_separator = "\n\n",
    output_key = "answer",
    return_source_documents = True,
    return_final_only = False,
    output_parser=StrOutputParser()
    # callbacks=[StdOutCallbackHandler()]
)

In [3]:
from langchain.callbacks.streaming_stdout import BaseCallbackHandler
from langchain.schema import Document
from typing import Sequence, Optional, Any
from uuid import UUID

# A benefit achieved from the langchain interface is that we can add callback handler also to the Retriever. 
# Without hard-coding the retrieved documents, such as filtering based on any condition,
# we can simpy use callback handler on_retriever_start, on_retriever_end, etc to perform anything we want.
# Below is just the example to print out the number of documents we retrieve through keylook.
class DocumentCallbackHandler(BaseCallbackHandler):
    def on_retriever_end(
        self,
        documents: Sequence[Document],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID]=None,
        **kwargs: Any
    ):
        print(f"on_retriever_end() called with {len(documents)} documents")
        
answer = chain("<YOUR_QUESTION>", callbacks=[DocumentCallbackHandler()])
print(f"__call__ output: {type(answer)}\noutput contains: {answer.keys()}")

on_retriever_end() called with 5 documents
__call__ output: <class 'dict'>
output contains: dict_keys(['question', 'context', 'answer', 'full_generation', 'source_documents'])


In [ ]:
# Instead of __call__ method, you can also use
# predict, apply, run (only when len(output_keys) == 1) just as LLMChain.
print(f"length of output_keys: {len(chain.output_keys)}")

answer = chain.predict(question="<YOUR_QUESTION>", callbacks=[DocumentCallbackHandler()])
print(f"predict output: {type(answer)}\noutput: {answer}")

In [ ]:
answer = chain.generate(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"generate output: {type(answer)}, \noutput: {answer}")

In [ ]:
answer = chain.apply(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"apply output: {type(answer)}, \noutput[0] contains: {answer[0]}")

In [ ]:
# It also supports async function.
answer = await chain.apredict(question="<YOUR_QUESTION>")
print(f"apredict output: {type(answer)}\noutput: {answer}")

answer = await chain.agenerate(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"agenerate output: {type(answer)}, \noutput: {answer}")

answer = await chain.aapply(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"aapply output: {type(answer)}, \noutput[0] contains: {answer[0]}")

## IRChain: with memory

In [14]:
from chains import KeylookChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.memory import ConversationBufferWindowMemory

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate.from_template(
    "Title {title}\n{page_content}"
)

system_template = """You are an assistant chatbot. Answer the best as you can."""

human_template = """Below are documents provided.
{context}

-------------
Previous conversation:
{chat_history}
Human: {question}
AI:"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

# Initialize llm and KeylookChain
# You can add streaming callbacks to llm model
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat_model = ChatOpenAI(
    openai_api_key = "<YOUR_KEY>",
    # streaming=True,
    # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# As KeylookChain inherits from LLMChain, it has the same interface as it.
# That means, you can add callbacks, output_parser, or any other modules
# provided through langchain.
from langchain.callbacks import StdOutCallbackHandler

memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)

chain = KeylookChain(
    memory = memory,
    prompt = prompt,
    document_prompt = document_prompt,
    llm = chat_model,
    retriever = retriever,
    document_variable_name = "context",
    document_separator = "\n\n",
    output_key = "answer",
    return_source_documents = True,
    return_final_only = False,
    output_parser=StrOutputParser(),
    # callbacks=[StdOutCallbackHandler()]
)

In [ ]:
answer = chain("<YOUR_QUESTION>", callbacks=[DocumentCallbackHandler()])
print(f"__call__ output: {type(answer)}\noutput contains: {answer.keys()}")

In [ ]:
# Instead of __call__ method, you can also use
# predict, apply, run (only when len(output_keys) == 1) just as LLMChain.
print(f"length of output_keys: {len(chain.output_keys)}")

answer = chain.predict(question="<YOUR_QUESTION>", callbacks=[DocumentCallbackHandler()])
print(f"predict output: {type(answer)}\noutput: {answer}")

In [ ]:
answer = chain.generate(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"generate output: {type(answer)}, \noutput: {answer}")

In [ ]:
answer = chain.apply(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"apply output: {type(answer)}, \noutput[0] contains: {answer[0]}")

In [ ]:
# It also supports async function.
answer = await chain.apredict(question="<YOUR_QUESTION>")
print(f"apredict output: {type(answer)}\noutput: {answer}")

answer = await chain.agenerate(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"agenerate output: {type(answer)}, \noutput: {answer}")

answer = await chain.aapply(
    [
        {"question":"<YOUR_QUESTION_1>"},
        {"question":"<YOUR_QUESTION_2>"}
    ]
)
print(f"aapply output: {type(answer)}, \noutput[0] contains: {answer[0]}")